[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab4/lab4.ipynb)
# Práctica 4: Redes neuronales con Regularización
## El sobreajuste (overfitting)
El problema del sobreajuste (*overfitting*) consiste en que la solución aprendida se ajusta muy bien a los datos de entrenamiento, pero no generaliza adecuadamente ante la aparición de nuevos datos. 

Para observar el sobreajuste en un problema, revisitaremos el problema de clasificación con datos del Titanic que hemos utilizado en los anteriores laboratorios. En primer lugar, carga los datos tal como se hacía en los anteriores notebooks. Asegúrate de tener una partición de entrenamiento con el 50\% de los datos y una de test con el 25\% y una de validación con el 25\% restante. Utiliza la función [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de `sklearn` sobre el `ndarray` de datos en dos veces sucesivas para conseguir las particiones. Después pasa los conjuntos resultantes a `torch` y envuélvelos en un `Dataloader` como se hacía en el laboratorio anterior.

**IMPORTANTE:** La separación de conjuntos debe hacerse sobre los datos sin preprocesar. Todo preprocesado se debe hacer sobre cada conjunto por separado, pero utilizando solo información del conjunto de entrenamiento. **Revisa la carga de datos de Titanic** para que la estandarización (paso 4 de la función de carga de datos) se realice después de la partición. La función devolverá los datos sin escalar y, tras hacer la separación en train/test/val, utilizaremos un único `StandardScaler` con el que haremos `fit_transform` sobre train y `transform` sobre test y val. Así evitaremos que se filtre ningún tipo de información de test/val al entrenamiento.

In [ ]:
# TODO - Carga el dataset Titanic en tres dataloaders: train/test/val

Una vez hayas cargado los datos, vamos a diseñar dos funciones para entrenar y evaluar un modelo. Las funciones deben tener esta forma:
- `def measure_model(model:nn.Module, loss_fn:nn.modules.loss._Loss, data_loader:data.DataLoader) -> tuple[float, float]:`
    - Devuelve una tupla con la pérdida y la accuracy en el conjunto pasado como parámetro. Asegúrate de poner el modelo en modo `eval`.
- `def train_model(model:nn.Module,
                optimizer:optim.Optimizer,
                train_loader:data.DataLoader,
                loss_fn:nn.modules.loss._Loss,
                num_epochs:int,
                val_loader:data.DataLoader,
                patience:None|int=None) -> tuple[tuple[list[int], list[int]]:`
    - Entrena el modelo con los datos de `train_loader` y devuelve una tupla con el histórico de losses en `train_loader` y en `val_loader`. Asegúrate de poner el modelo en modo `train`.

In [ ]:
def measure_model(model:nn.Module, loss_fn:nn.modules.loss._Loss, data_loader:data.DataLoader) -> tuple[float, float]:
    #TODO Completa la función

def train_model(model:nn.Module,
                optimizer:optim.Optimizer,
                train_loader:data.DataLoader,
                loss_fn:nn.modules.loss._Loss,
                num_epochs:int,
                val_loader:data.DataLoader=None,
                ) -> tuple[tuple[list[int], list[int]]:

    #TODO Completa la función
        
    return train_losses, val_losses

Para este primer apartado, diseña una red neuronal totalmente conectada con tres capas ocultas de 20, 10 y 5 neuronas respectivamente.

In [ ]:
#TODO Define la subclase de nn.Module con la arquitectura descrita

Ahora entrena el modelo utilizando el optimizador `Adam` y la función de coste `BCELoss`. Tras entrenar, calcula la accuracy en test y muestra con `matplotlib` la curva de entrenamiento.

In [ ]:
model = ...
# print(model)

# Definimos la función de pérdida y el optimizador
loss_fn = ...
optimizer = ...

num_epochs = 700

train_losses, val_losses = ...

pyplot.plot(train_losses, label="train")
pyplot.plot(val_losses, label="val")
pyplot.legend()
pyplot.show()

test_loss, test_accuracy = ...
print(f'Test loss:{test_loss:.4f}\tTest accuracy:{test_accuracy:.2f}')

En la curva de entrenamiento deberías ser capaz de identificar que hay sobreentrenamiento porque la curva de validación llega a un punto que deja de mejorar y comienza a empeorar.

*NOTA: Puedes exagerar el efecto de sobreajuste tomando menos datos de entrenamiento, lo cual facilita su memorización.*

# Regularización
Una vez diagnosticado el sobreajuste, es hora de probar diferentes técnicas que intenten reducir la varianza, sin incrementar demasiado el sesgo y, con ello, el modelo generaliza mejor. Las técnicas de regularización que vamos a ver en este laboratorio son:
1. *Early stopping*. Detiene el entrenamiento de la red cuando aumenta el error.
1. Penalización basada	en	la	norma	de	los	parámetros (tanto norma L1 como L2). 
1. *Dropout*. Ampliamente utilizada en aprendizaje profundo, "desactiva" algunas neuronas para evitar el sobreajuste.

## Parte 1. Early Stopping
Hacer parada temprana (*early stopping*) consiste en monitorizar el rendimiento en un pequeño subconjunto del conjunto de entrenamiento y parar el aprendizaje cuando este rendimiento decaiga.

Para llevarlo a cabo, añade a la función de `train_model` un parámetro opcional llamado `patience` que indique durantos cuántos pasos puede empeorar el la pérdida en validación sin que se pare el entrenamiento. Cuando esté presente el valor de `patience` el entrenamiento se tiene que parar atendiendo a él.

Una vez tengas la función, vuelve a declarar el modelo y optimizador, entrena y revisa las curvas y el rendimiento en test.

In [ ]:
# TODO - Instancia un modelo y un optimizador, haz el entrenamiento con early stopping y muestra las curvas y medidas

# Parte 2. Penalización basada en norma de parámetros
Otra alternativa para regularizar consiste en añadir a la función de pérdida una componente que penalice los valores altos en los parámetros. En `torch` le podemos indicar al optimizador que incluya una penalización L2 en los pesos indicando el parámetro `weight_decay` al instanciarlo. Prueba distintos valores (suele ser buena política probar cambios en factores de 10 alrededor del 0.001) y comprueba su efecto en el rendimiento. No utilices *early stopping* en el entrenamiento.

**¿Qué diferencia observas en la curva de entrenamiento con respecto a utilizar *early stopping*?**

In [ ]:
# TODO - Instancia un modelo y un optimizador con weight decay, haz el entrenamiento y muestra las curvas y medidas

# Parte 3. Dropout
Por último, la tercera alternativa que exploraremos para regularizar es añadir capas `nn.Dropout` a nuestra arquitectura. Estas capas apagan durante el entrenamiento una fracción (que debemos indicar como parámetro al instanciar la capa) de las neuronas que reciben a la entrada, impidiendo al modelo apoyarse siempre en las mismas entradas para hacer sus predicciones, forzándolo así a aprender representaciones más generales y, por tanto, mejorando su generalización.

Crea una nueva arquitectura de red con estas capas y entrénala (sin usar ningún otro mecanismo de regularización). Prueba a situar las capas `nn.Dropout` en distintos sitios y a aplicar distintos valores del parámetro `p` que indica la probabilidad de que se apague cada neurona.

In [ ]:

# TODO - Declara la nueva clase, haz que tu modelo sea una instancia de ella, instancia el optimizador, haz el entrenamiento y muestra las curvas y medidas.